## Hypothesis testing and A/B Testing details

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

A very useful [visualization](https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/hypothesis-testing/selecting-a-hypothesis-test.html) on which hypothesis test to use

<img src="Flowchart.svg" style="width 200px">

### Determining sample size
A commonly asked question in this scenario is how many samples do we need to have 80% power to reject the null hypothesis that this treatment does not do much?

For this we need the **BEAN** parameters:
* **B**: Beta error, and power = 1-beta error (beta error = probability of false negative)
* **E**: Effect size, ie, difference between the two groups with respect to the standard deviation, $\frac{\mu_1-\mu_2}{\sigma}$
* **A**: $\alpha$ error rate, or probability of false positive
* **N**: Sample size

Imagine the distribution of the null hypothesis with no difference between groups, ie, 

$$H_0: X \sim N[0,\sigma]$$
$$H_1: X \sim N[\mu_1 - \mu_2,\sigma]$$

where $\mu_1$ and $\mu_2$ are the means of the control and treatment groups and $\sigma$ is the standard deviation of the mean in estimation. 

$Z_{\alpha}$ is the value of the standard normal that will determine whether or not we reject the null hypothesis. So in the distribution of the treatment group, any samples < $Z_{\alpha}$ will be rejected. To have enough power $pow$, we need this $Z_{\alpha}$ to be less than the $X$ required to get the power, $Z_{1-pow}$ from the distribution of the alternate hypothesis

To derieve this, we know $X_{\alpha} = Z_{\alpha}*\frac{\sigma}{\sqrt{N}}$ and $X_{pow} = Z_{1-pow}*\frac{\sigma}{\sqrt{N}} + \mu$

When the two $X$'s are equal, ie, $X_{\alpha} = X_{pow}$, we have $$n = (\frac{Z_{\alpha} - Z_{1-pow}}{\frac{\mu}{\sigma}})^2$$

$Z_{1-pow}$ is usually negative and $Z_{\alpha}$ is usually positive

#### Case 1: Discrete data with a normal approximation of a binomial distribution

Let's say we are interested in knowing how many times will an event happen if we did an experiment $N$ times? Each event has a probability $p$ of happening and $1-p$ of not happening. Then what is the ensemble estimate of the event happening?

A classic example of this is in A/B testing in software companies that is aimed to determine if a certain treatment improved the click-through probability of a button, e.g., would changing the color of a button change the probability of it being clicked

Let's say we did an experiment with $N=1000$ people and found that $X=100$ clicked it. That means that in our treatment population:
* there is a mean $\hat{p} = 0.1 = 100/1000$ probability of the button being clicked
* STD of the number of clicks is $\sqrt{N*p*(1-p)}$, and the STD of the click probability is $\frac{\sqrt{N*p*(1-p)}}{N}$
* $\sigma = \frac{\sqrt{p*(1-p)}}{\sqrt{N}}$
* The ensemble distribution is a normal distribution with $N(\hat{p}, \sigma)$

**Example problem 1:** Nixon thinks that he’s a better at the game “rock, paper, scissors” than his friend Kissinger. To find out if this is the case, he challenges Kissinger to 49 bouts of rock, paper, scissors. Nixon wins 31 of these bouts. Can we reject the null
hypothesis that the two men are equally good at the game (i.e., P(Nixon
wins)=P(Nixon loses)=.5) at an alpha level of .05?

**Solution**: Null hypothesis that Nixon and Kissinger are equally good = (P(Nixon wins) = 0.5) = $N[0.5, \sigma_p]$

where, $\sigma_p = \frac{\sqrt{p*(1-p)}}{\sqrt N} $

In [35]:
N, w=50, 31
p = w/N
base_p = 0.5
Z_alpha = stats.norm.ppf(0.95); 

sigma_p = np.sqrt(p*(1-p))/np.sqrt(N); print(f'$\sigma$ = {sigma_p:0.2f}')

Z_p = (p - base_p)/sigma_p; 

print(f'Z_alpha={Z_alpha:0.3f}')
print(f'Z_p={Z_p:0.3f}')

print('Since Z_p < Z_alpha, we can reject the null hypothesis')

$\sigma$ = 0.07
Z_alpha=1.645
Z_p=1.748
Since Z_p < Z_alpha, we can reject the null hypothesis


**Example problem 2:** You are running for office and your pollster polled hundred people. Sixty of them claimed they will vote for you. Can you relax?

**Solution**: There is an error here but I don't know why

Question is #32 [here](https://towardsdatascience.com/40-statistics-interview-problems-and-answers-for-data-scientists-6971a02b7eee)

In [43]:
N, yes = 100, 60
p, base_p = yes/N, 0.5
sigma = np.sqrt(p*(1-p))/np.sqrt(N)

Z_p = (p - base_p)/sigma

Z_alpha2 = stats.norm.ppf(0.975)

print(f'Z_p: {Z_p:0.2f}'); 
print(f'Z_alpha two-sided ={Z_alpha2:0.3f}')
print('Since Z_p > Z_alpha2, we cannot be sure this is a significant result, so no relaxing just yet')


Z_p: 2.04
Z_alpha two-sided =1.960
Since Z_p > Z_alpha2, we cannot be sure this is a significant result, so no relaxing just yet


#### Case 2: Discrete data with a poisson distribution

The Normal approximation still holds, with the following parameters:

$variance = N, \sigma = \sqrt(N)$

**Example Problem 3**: The homicide rate in Scotland fell last year to 99 from 115 the year before. Is this reported change really noteworthy

In [49]:
Nprev, Ncurr = 115, 99
sigma = np.sqrt(Nprev);

Z_p = (Nprev - Ncurr)/sigma; 

print(f'Z_p = {Z_p:0.2f}')
print('Since this value is less than the Z-score from a 2tailed comparison, we cannot reject the null hypothesis')

Z_p = 1.49
Since this value is less than the Z-score from a 2tailed comparison, we cannot reject the null hypothesis


### Hypothesis testing with independent samples

#### Case 3: Binomial proportions

If the two samples are independent, such as a Control and a Treatment group, we need to do an independent samples analysis. For this we need to calculate some pooled variables:

For two groups with $n_1$ and $n_2$ people in each, let's say $x_1$ and $x_2$ clicked a button. The pooled probability $p_{pool}$ is calculated as $$p_{pool} = \frac{x_1+x_2}{n_1+n_2}$$ and the pooled standard deviation as $$s_p = \sqrt{p_{pool}*(1-p_{pool})*(\frac{1}{n_1} + \frac{1}{n_2})}$$

And the Z-statistic is $$Z = \frac{p_1-p_2}{s_p}$$

All downstream calculations are the same. This is [taken from here](https://online.stat.psu.edu/stat800/lesson/5/5.5)

**Example problem:** Company X manufactures laptops. For quality control, two sets of laptops were tested. In the first group, 32 out of 800 had a defect. In the other group, 30 out of 500 were defective. Is there a difference between the two groups (with alpha of 0.05)?

In [55]:
X1,X2,N1,N2 = 32,30,800,500
p1,p2 = X1/N1, X2/N2
p_pool = (X1+X2)/(N1+N2);
s_pool = np.sqrt(p_pool*(1-p_pool)*(1/N1 + 1/N2))
Z = np.abs(p1-p2)/s_pool; print(Z)
print('Z < 1.96 so there is no significant difference between the two groups')

1.6461730990737975
Z < 1.96 so there is no significant difference between the two groups


#### Case 4: Normal distribution:

For hypothesis with independent samples in two groups (1&2), the formula for pooled standard deviation is different. Keep in mind that this is valid only if the variances in the two groups are similar. As a rool of thumb, if $0.5<=\frac{\sigma_1^2}{\sigma_2^2}<=2$, this formula holds
$$s_p = \sqrt{\frac{(n_1-1)*\sigma_1+(n_2-1)*\sigma_2}{n_1+n_2-2}}$$

and the effective $n_{eff}$ is 
$$\frac{1}{n_{eff}} = \frac{1}{n_1} + \frac{1}{n_2}$$

**Example problem**: [From here](https://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/BS704_Confidence_Intervals/BS704_Confidence_Intervals5.html)

In [61]:
n1, n2 = 1623, 1911
s1, s2 = 17.5, 20.1
x1, x2 = 128.2, 126.5
neff = np.sqrt(1/n1 + 1/n2)

s = np.sqrt((n1-1)*s1**2 + (n2-1)*s2**2)/np.sqrt(n1+n2-2)

ci_95 = [x1-x2 - 1.96*s*neff, x1-x2 + 1.96*s*neff] 
print(ci_95)

[0.446232731732771, 2.9537672682672063]


This is a [good resource](https://sphweb.bumc.bu.edu/otlt/mph-modules/bs/bs704_power/bs704_power_print.html) as well 